<a href="https://colab.research.google.com/github/TaniaHurtado2024/Seminario_BigData/blob/main/1_PPvsSpark_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas vs Polars vs Spark vs Dask
[Author: Elias Buitrago Bolivar](https://github.com/ebuitrago?tab=repositories)

Inspired in: https://www.youtube.com/watch?v=mi9f9zOaqM8

Original data: Kaggle

This jupyter notebook is designed to study and compare different tools to read and manipulate data; to be used in the data undertanding phase. The corresponding explanations will be given directly in class, therefore the material isn't autoexplained. Don´t forget ask me for the access to the data. And, please, give credits to the original author's idea and, if consider, also to me.

_Updated: June 20th, 2023_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Playing with pandas

In [ ]:
import pandas as pd
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = pd.read_parquet(flights_file1)
df2 = pd.read_parquet(flights_file2)
#df3 = pd.read_parquet(flights_file3)
df4 = pd.read_parquet(flights_file4)
#df5 = pd.read_parquet(flights_file5)

In [ ]:
df = pd.concat([df1, df2, df4])
#df = df1

In [ ]:
%%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_pandas.parquet")

In [ ]:
!ls -GFlash temp_pandas.parquet

In [ ]:
pd.read_parquet('temp_pandas.parquet')

In [ ]:
pd.read_parquet('temp_pandas.parquet').info()

## Playing with Polars

In [4]:
import polars as pl

In [1]:
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = pl.scan_parquet(flights_file1)
df2 = pl.scan_parquet(flights_file2)
#df3 = pl.scan_parquet(flights_file3)
df4 = pl.scan_parquet(flights_file4)
#df5 = pl.scan_parquet(flights_file5)

NameError: name 'pl' is not defined

In [6]:
 %%timeit

df_polars = (
    pl.concat([df1, df2, df4])
    .groupby(['Airline', 'Year'])
    .agg([
        pl.col("DepDelayMinutes").mean().alias("avg_dep_delay"),
        pl.col("DepDelayMinutes").sum().alias("sum_dep_delay"),
        pl.col("DepDelayMinutes").max().alias("max_dep_delay"),
        pl.col("ArrDelayMinutes").mean().alias("avg_arr_delay"),
        pl.col("ArrDelayMinutes").sum().alias("sum_arr_delay"),
        pl.col("ArrDelayMinutes").max().alias("max_arr_delay"),
      ])
).collect()

df_polars.write_parquet('temp_polars.parquet')

<magic-timeit>:4: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.


1.82 s ± 450 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
!ls -GFlash temp_polars.parquet

8.0K -rw-r--r-- 1 root 4.6K Jul 13 20:57 temp_polars.parquet


## Playing with PySpark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2f594e9d45f00b4709aa93c676e8037ded1c6e55374124d1a860eddda8c0ec1a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, sum, concat

In [4]:
spark = SparkSession.builder.master("local[1]").appName("airline-example").getOrCreate()

In [5]:
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"

In [6]:
df_spark1 = spark.read.parquet(flights_file1)
df_spark2 = spark.read.parquet(flights_file2)
#df_spark3 = spark.read.parquet(flights_file3)
df_spark4 = spark.read.parquet(flights_file4)
#df_spark5 = spark.read.parquet(flights_file5)

In [7]:
df_spark = df_spark1.union(df_spark2)
#df_spark = df_spark.union(df_spark3)
df_spark = df_spark.union(df_spark4)
#df_spark = df_spark.union(df_spark5)

In [8]:
 %%timeit

df_spark_agg = df_spark.groupby("Airline", "Year").agg(
    avg("ArrDelayMinutes").alias('avg_arr_delay'),
    sum("ArrDelayMinutes").alias('sum_arr_delay'),
    max("ArrDelayMinutes").alias('max_arr_delay'),
    avg("DepDelayMinutes").alias('avg_dep_delay'),
    sum("DepDelayMinutes").alias('sum_dep_delay'),
    max("DepDelayMinutes").alias('max_dep_delay'),
)
df_spark_agg.write.mode('overwrite').parquet('temp_spark.parquet')

2.64 s ± 742 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
!ls -GFlash temp_pyspark.parquet

ls: cannot access 'temp_pyspark.parquet': No such file or directory


## Playing with dask

In [4]:
import pandas as pd
import dask.dataframe as dd
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
#flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = dd.read_parquet(flights_file1)
df2 = dd.read_parquet(flights_file2)
#df3 = dd.read_parquet(flights_file3)
df4 = dd.read_parquet(flights_file4)
#df5 = dd.read_parquet(flights_file5)

In [5]:
df = dd.concat([df1,df2,df4])

In [6]:
print(df.compute())

       FlightDate                      Airline Origin Dest  Cancelled  \
0      2018-01-23            Endeavor Air Inc.    ABY  ATL      False   
1      2018-01-24            Endeavor Air Inc.    ABY  ATL      False   
2      2018-01-25            Endeavor Air Inc.    ABY  ATL      False   
3      2018-01-26            Endeavor Air Inc.    ABY  ATL      False   
4      2018-01-27            Endeavor Air Inc.    ABY  ATL      False   
...           ...                          ...    ...  ...        ...   
637609 2018-09-11  Air Wisconsin Airlines Corp    SCE  IAD      False   
637610 2018-09-11  Air Wisconsin Airlines Corp    IAD  GSO      False   
637611 2018-09-11  Air Wisconsin Airlines Corp    EVV  ORD      False   
637612 2018-09-11  Air Wisconsin Airlines Corp    ORD  HPN      False   
637613 2018-09-11  Air Wisconsin Airlines Corp    HPN  ORD      False   

        Diverted  CRSDepTime  DepTime  DepDelayMinutes  DepDelay  ...  \
0          False        1202   1157.0             

In [7]:
df = df.compute()

In [8]:
%%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_dask.parquet")

924 ms ± 167 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
!ls -GFlash temp_dask.parquet

12K -rw-r--r-- 1 root 9.2K Jul 13 21:22 temp_dask.parquet


In [10]:
pd.read_parquet('temp_dask.parquet').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (Airline, )              28 non-null     string 
 1   (Year, )                 28 non-null     int64  
 2   (DepDelayMinutes, mean)  28 non-null     float64
 3   (DepDelayMinutes, sum)   28 non-null     float64
 4   (DepDelayMinutes, max)   28 non-null     float64
 5   (ArrDelayMinutes, mean)  28 non-null     float64
 6   (ArrDelayMinutes, sum)   28 non-null     float64
 7   (ArrDelayMinutes, max)   28 non-null     float64
dtypes: float64(6), int64(1), string(1)
memory usage: 1.9 KB


In [11]:
pd.read_parquet('temp_dask.parquet')

Airline  Year DepDelayMinutes  \
                                                               mean   
0                 Air Wisconsin Airlines Corp  2018       16.753459   
1                        Alaska Airlines Inc.  2018        7.503389   
2                               Allegiant Air  2018       17.080944   
3                      American Airlines Inc.  2018       13.141112   
4                                    Cape Air  2018        4.643761   
5                 Capital Cargo International  2018       14.876462   
6                                 Comair Inc.  2018       12.776783   
7   Commutair Aka Champlain Enterprises, Inc.  2018       28.243923   
8                            Compass Airlines  2018       14.060415   
9                        Delta Air Lines Inc.  2018        8.538123   
10                       Empire Airlines Inc.  2018       13.654324   
11                          Endeavor Air Inc.  2018       13.952577   
12                                  Envoy Air  2018       10.910172   
13                   ExpressJet Airlines Inc.  2018       16.544919   
14                     Frontier Airlines Inc.  2018       23.029903   
15   GoJet Airlines, LLC d/b/a United Express  2018       16.427071   
16                     Hawaiian Airlines Inc.  2018        5.011211   
17                                Horizon Air  2018        7.285063   
18                            JetBlue Airways  2018       19.670430   
19                         Mesa Airlines Inc.  2018       15.128549   
20                     Peninsula Airways Inc.  2018       14.184256   
21                          Republic Airlines  2018       12.002701   
22                      SkyWest Airlines Inc.  2018       14.514365   
23                     Southwest Airlines Co.  2018       12.598293   
24                           Spirit Air Lines  2018       13.666850   
25                      Trans States Airlines  2018       21.379592   
26                      United Air Lines Inc.  2018       13.614583   
27                             Virgin America  2018       10.902274   

                       ArrDelayMinutes                      
           sum     max            mean         sum     max  
0    1606774.0  1296.0       17.881934   1708887.0  1292.0  
1    1374801.0   839.0        8.759125   1600336.0   842.0  
2    1630769.0  1462.0       17.547588   1670390.0  1505.0  
3    4993399.0  2109.0       14.225643   5387564.0  2153.0  
4       7704.0   430.0        5.390332      8921.0   446.0  
5     625823.0   841.0       15.310871    640270.0   814.0  
6    1452158.0  1121.0       12.789146   1447872.0  1110.0  
7    1290832.0  1352.0       29.284076   1332689.0  1353.0  
8     629302.0  2625.0       14.836996    662917.0  2635.0  
9    3924514.0  1207.0        8.368956   3840012.0  1206.0  
10    115420.0   655.0       14.458483    120497.0   654.0  
11   1629061.0  1921.0       14.456739   1691424.0  1916.0  
12   1353516.0  1163.0       12.531226   1546817.0  1152.0  
13   2689873.0  1522.0       17.858140   2892215.0  1553.0  
14   2711702.0  1254.0       22.355794   2627320.0  1253.0  
15   1029156.0  1545.0       16.987283   1060635.0  1543.0  
16    418366.0  2482.0        5.944504    495623.0  2475.0  
17    613854.0   640.0        8.279781    694980.0   642.0  
18   5876305.0  1489.0       19.820096   5901592.0  1473.0  
19   2274532.0  1789.0       16.418326   2460122.0  1773.0  
20     16397.0   470.0       15.569902     17485.0   437.0  
21   2377663.0  1270.0       13.293485   2624772.0  1346.0  
22   7611841.0  2098.0       15.656154   8173859.0  2108.0  
23  16812158.0   713.0       11.119075  14806261.0   698.0  
24   2385002.0  1527.0       14.241442   2479848.0  1527.0  
25   1389310.0  1519.0       22.335766   1444320.0  1537.0  
26   8398591.0  1431.0       14.477765   8902595.0  1429.0  
27    187977.0   520.0       11.950971    204995.0   504.0

## Read Results

In [ ]:
import pandas as pd

In [ ]:
#agg_pandas = pd.read_parquet('temp_pandas.parquet')
agg_polars = pd.read_parquet('temp_polars.parquet')
agg_spark = pd.read_parquet('temp_spark.parquet')
agg_dask = pd.read_parquet('temp_dask.parquet')

In [ ]:
#agg_pandas.shape, agg_polars.shape, agg_spark.shape, agg_dask.shape

In [ ]:
#agg_pandas.sort_values(['Airline','Year']).head()

In [ ]:
agg_polars.sort_values(['Airline','Year']).head()

,Airline,Year,avg_dep_delay,sum_dep_delay,max_dep_delay,avg_arr_delay,sum_arr_delay,max_arr_delay
5,Air Wisconsin Airlines Corp,2018,16.753459,1606774.0,1296.0,17.881934,1708887.0,1292.0
97,Air Wisconsin Airlines Corp,2019,16.868511,1742281.0,1690.0,17.610384,1811545.0,1707.0
51,Air Wisconsin Airlines Corp,2020,8.583725,433315.0,1460.0,8.982529,452450.0,1439.0
49,Air Wisconsin Airlines Corp,2021,16.553045,1290194.0,1421.0,17.327440,1346602.0,1416.0
106,Air Wisconsin Airlines Corp,2022,13.124801,510581.0,1355.0,13.340409,517261.0,1353.0


In [ ]:
agg_spark.sort_values(['Airline','Year']).head()

,Airline,Year,avg_arr_delay,sum_arr_delay,max_arr_delay,avg_dep_delay,sum_dep_delay,max_dep_delay
0,Air Wisconsin Airlines Corp,2018,17.881934,1708887.0,1292.0,16.753459,1606774.0,1296.0
48,Air Wisconsin Airlines Corp,2019,17.610384,1811545.0,1707.0,16.868511,1742281.0,1690.0
56,Air Wisconsin Airlines Corp,2020,8.982529,452450.0,1439.0,8.583725,433315.0,1460.0
93,Air Wisconsin Airlines Corp,2021,17.327440,1346602.0,1416.0,16.553045,1290194.0,1421.0
119,Air Wisconsin Airlines Corp,2022,13.340409,517261.0,1353.0,13.124801,510581.0,1355.0


In [ ]:
agg_dask.sort_values(['Airline','Year']).head()

Airline  Year DepDelayMinutes                     \
                                                mean        sum     max   
0  Air Wisconsin Airlines Corp  2020        8.583725   433315.0  1460.0   
1  Air Wisconsin Airlines Corp  2022       13.124801   510581.0  1355.0   
2         Alaska Airlines Inc.  2020        5.818328   772930.0   823.0   
3         Alaska Airlines Inc.  2022       10.153994  1278134.0   915.0   
4                Allegiant Air  2020       12.825575  1080016.0  1648.0   

  ArrDelayMinutes                     
             mean        sum     max  
0        8.982529   452450.0  1439.0  
1       13.340409   517261.0  1353.0  
2        6.365082   843157.0   788.0  
3       11.026280  1382905.0   908.0  
4       13.331111  1115734.0  1645.0